# Creates Shiller Investment Dataset
- Converts Shiller spreadsheet to Shiller Investment Dataset.csv   
    - Selects specific factors
    - Adds stock and bond returns 

Dependencies   
- http://www.econ.yale.edu/~shiller/data/ie_data.xls


In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

## Import Shiller spreadsheet

In [2]:
link = 'http://www.econ.yale.edu/~shiller/data/ie_data.xls'

In [3]:
xls = pd.ExcelFile(link)
df = pd.read_excel(xls,'Data', header = 7)
df.tail()

,Date,P,D,E,CPI,Fraction,Rate GS10,Price,Dividend,Price.1,...,CAPE,Unnamed: 13,TR CAPE,Unnamed: 15,Yield,Returns,Returns.1,Real Return,Real Return.1,Returns.2
1814,2022.03,4391.27,61.969974,197.89,287.504,2022.208333,2.13,4488.795770,63.346335,2.883798e+06,...,34.270844,NaN,37.239465,NaN,0.030717,0.948016,47.304227,NaN,NaN,NaN
1815,2022.04,4391.3,NaN,NaN,289.109,2022.291667,2.75,4463.907335,NaN,2.867809e+06,...,33.889253,NaN,36.757253,NaN,0.025106,0.989378,44.596200,NaN,NaN,NaN
1816,2022.05,4040.36,NaN,NaN,292.296,2022.375000,2.9,4062.386691,NaN,2.609854e+06,...,30.803849,NaN,33.386294,NaN,0.027804,0.953019,43.641414,NaN,NaN,NaN
1817,2022.06,3789.99,NaN,NaN,293.889,2022.458333,3.49,3789.990000,NaN,2.434855e+06,...,28.702967,NaN,31.086592,NaN,0.024988,NaN,41.365606,NaN,NaN,NaN
1818,NaN,June 2022 P is June 15 close,NaN,S&P Estimated Earnnings,June 2022 CPI estimated,NaN,June 2022 GS10 is June 14 value,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['Date'] = pd.date_range(start='1/1/1871', periods=len(df), freq='M')    


### Rename Factors

In [5]:
df = df.rename({'Date': 'Date', 'P': 'Price_nom', 'D': 'Div_nom', 'E': 'Earn_nom', 'Dividend': 'Div_real'}, axis=1) 


df = df.rename({'Rate GS10': 'Yield_10yr', 'Price': 'Price_real', 'Price.1': 'Price_TR_real', 'Dividend': 'Div_real'}, axis=1) 


df = df.rename({'Earnings': 'Earn_real', 'Earnings.1': 'Earn_TR_real', 'TR CAPE': 'CAPE_TR'}, axis=1) 


df = df.rename({'Yield': 'Excess_CAPE_yld', 'Returns': 'Bond_ret_nom', 'Returns.1': 'Bond_ret_real'}, axis=1) 


df = df.rename({'Real Return': 'Stock_10ret_real', 'Real Return.1': 'Bond_10ret_real', 'Returns.2': 'Excess_10ret_real'}, axis=1) 
df.columns

Index(['Date', 'Price_nom', 'Div_nom', 'Earn_nom', 'CPI', 'Fraction',
       'Yield_10yr', 'Price_real', 'Div_real', 'Price_TR_real', 'Earn_real',
       'Earn_TR_real', 'CAPE', 'Unnamed: 13', 'CAPE_TR', 'Unnamed: 15',
       'Excess_CAPE_yld', 'Bond_ret_nom', 'Bond_ret_real', 'Stock_10ret_real',
       'Bond_10ret_real', 'Excess_10ret_real'],
      dtype='object')

### Drop columns

In [6]:
df.drop(['Unnamed: 13', 'Unnamed: 15'], inplace=True, axis=1)

### Create New Dataframe With Specific Factors

In [7]:
df1 = df[['Date', 'Price_nom', 'Div_nom', 'Earn_nom', 'Yield_10yr', 'CPI']]
df1.tail()

,Date,Price_nom,Div_nom,Earn_nom,Yield_10yr,CPI
1814,2022-03-31,4391.27,61.969974,197.89,2.13,287.504
1815,2022-04-30,4391.3,NaN,NaN,2.75,289.109
1816,2022-05-31,4040.36,NaN,NaN,2.9,292.296
1817,2022-06-30,3789.99,NaN,NaN,3.49,293.889
1818,2022-07-31,June 2022 P is June 15 close,NaN,S&P Estimated Earnnings,June 2022 GS10 is June 14 value,June 2022 CPI estimated


### Drop rows with NaN's

In [8]:
df1 = df1.dropna()

### Change data types

In [9]:
df1["Date"] = pd.to_datetime(df1["Date"])
df1["Price_nom"] = pd.to_numeric(df1["Price_nom"])
df1["Div_nom"] = pd.to_numeric(df1["Div_nom"])
df1["Earn_nom"] = pd.to_numeric(df1["Earn_nom"])
df1["Yield_10yr"] = pd.to_numeric(df1["Yield_10yr"])
df1["CPI"] = pd.to_numeric(df1["CPI"])

In [10]:
df1.dtypes

Date          datetime64[ns]
Price_nom            float64
Div_nom              float64
Earn_nom             float64
Yield_10yr           float64
CPI                  float64
dtype: object

### Calculate Stock Return

In [11]:

df1["Stk_ret"] = (df1.Price_nom+df1.Div_nom/12)/df1.Price_nom.shift()-1

### Calculate Bond Return

In [12]:
# Yields
YTM0 = df1.Yield_10yr.shift()
YTM1 = df1.Yield_10yr


In [13]:
pmt=0
for i in range(10):
    pmt = pmt + (YTM0/100)/(1+YTM1/100)**((i+11/12))
P1 = pmt + 1/(1+YTM1/100)**(9+11/12)
df1["Bnd_ret"] = P1-1

### Calculate Dividend Yield

In [14]:
df1["Div_yield"] = df1.Div_nom/df1.Price_nom.shift()

### Set Start Date at 12/1919 and End Date 12/2021

In [15]:
df1 = df1[df1.Date > "1919-12-1"]
df1 = df1[df1.Date < "2022-1-1"]

In [16]:
df1.describe()

,Price_nom,Div_nom,Earn_nom,Yield_10yr,CPI,Stk_ret,Bond_ret,Div_yield
count,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000,1225.000000
mean,489.773891,10.061741,23.304008,4.783199,88.199781,0.009299,0.004183,0.038793
std,808.010099,14.056567,34.628441,2.728199,81.585873,0.044050,0.014879,0.017425
min,4.770000,0.440000,0.290000,0.620000,12.600000,-0.261879,-0.080542,0.011007
25%,16.890000,0.850000,1.290000,2.661667,19.000000,-0.010116,-0.000829,0.024289
50%,93.010000,3.110000,5.736670,3.960000,39.800000,0.012864,0.003031,0.036364
75%,661.230000,14.270000,27.330000,6.270000,156.700000,0.032581,0.008938,0.050794
max,4674.772727,60.397117,197.870000,15.320000,278.802000,0.513085,0.105548,0.132767


In [17]:
df1.set_index('Date', inplace=True)

### Save As 'Shiller Investment Dataset.csv'

In [18]:
df1.to_csv('Shiller Investment Dataset.csv', index=True) 

#### Check dataset

In [19]:
df2 = pd.read_csv (r'Shiller Investment Dataset.csv')

In [20]:
df2

,Date,Price_nom,Div_nom,Earn_nom,Yield_10yr,CPI,Stk_ret,Bond_ret,Div_yield
0,1919-12-31,8.920000,0.530000,0.930000,4.930833,18.900,-0.024574,0.000972,0.057671
1,1920-01-31,8.830000,0.528300,0.919200,4.970000,19.300,-0.005154,0.001009,0.059226
2,1920-02-29,8.100000,0.526700,0.908300,4.980000,19.500,-0.077702,0.003282,0.059649
3,1920-03-31,8.670000,0.525000,0.897500,4.990000,19.700,0.075772,0.003290,0.064815
4,1920-04-30,8.600000,0.523300,0.886700,5.000000,20.300,-0.003044,0.003299,0.060358
...,...,...,...,...,...,...,...,...,...
1220,2021-08-31,4454.206364,58.791336,169.873333,1.280000,273.567,0.021860,0.004797,0.013473
1221,2021-09-30,4445.543333,59.254483,175.430000,1.370000,274.310,-0.000836,-0.007232,0.013303
1222,2021-10-31,4460.707143,59.635361,182.910000,1.580000,276.589,0.004529,-0.018003,0.013415
1223,2021-11-30,4667.386667,60.016239,190.390000,1.560000,277.948,0.047455,0.003132,0.013454
